In [1]:
import tensorflow as tf
from keras.layers import Dense, LSTM, Activation, Bidirectional, Dropout, Input, concatenate
from keras.models import Sequential
from keras import Model
import keras.backend as K

/Users/marcusdaly/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/marcusdaly/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/marcusdaly/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/marcusdaly/anaconda3/lib/python3.6/site-packages/tensorflow/

In [ ]:

n_dims = 50 # len(n)
z_dims = 50 # len(Z)
total_latent_dims = n_dims+z_dims # len(N) + len(Z)

In [ ]:
def squareError(xTrue, xPred):
    return K.square(xTrue - xPred)


In [ ]:
def reconstructionLoss(sample, encoder, decoder, f_w, weight): # (L_1 from the paper)
    return K.mean(squareError(sample, decoder(tf.concat(encoder(sample), axis=1))) + 
                  weight*K.log(f_w(tf.concat(encoder(sample), axis=1))), axis=0)


In [ ]:
def divergenceLoss(f_w, encoder, sample, z_j, n_j): # Mean of log f_w(E_theta_i(x_j)) + log (1-f_w(z_j, n_j)) from the paper (L_2).
    return K.mean(K.log(f_w(tf.concat(encoder(sample), axis=1))) + 
                  K.log(1 - f_w(tf.concat([z_j, n_j], axis=1))), axis=0)


In [ ]:
# Currently just doing a restriction to the last z variables, might want to do a matrix multiplication?
# pi_Z from the paper. projects a latent distribution in (z, n) to z
def projectZ(encoded):
    return encoded[0] # take zs.

In [ ]:
# takes in two inputs, n and z, and outputs samples.
def createDecoder(z_dims, n_dims, output_dims):
    # TODO MAYBE: Add in more regularization or different than dropout?

    z_inputs = Input(shape=(z_dims))
    n_inputs = Input(shape=(n_dims))
    inputs = concatenate([z_inputs, n_inputs])
    # 150 is arbitrary rn...
    dense = Dense(150)(inputs)
    bilstm = Bidirectional(LSTM(32, activation='tanh', return_sequences=True))(dense)
    bilstm = Dropout(0.2)(bilstm)
    outputs = Bidirectional(LSTM(32, activation='tanh', dropout=0.2, return_sequences=True))(inputs)
    
    model = Model(inputs=[z_inputs, n_inputs], outputs=outputs)
    
    model.compile()
    return model

In [ ]:
def createEncoder(input_num, z_dims, n_dims):
    # TODO MAYBE: Add in more regularization or different than dropout?
    inputs = Input(shape=(time_steps, features))
    bilstm = Bidirectional(LSTM(32, activation='tanh', return_sequences=True))(bilstm)
    bilstm = Dropout(0.2)(bilstm)
    dense = Bidirectional(LSTM(32, activation='tanh', dropout=0.2, return_sequences=False))(bilstm)
    dense = Dropout(0.5)(dense)
    z_output = Dense(z_dims)(dense)
    n_output = Dense(n_dims)(dense)
    
    model = Model(inputs=inputs, outputs=[z_output, n_output])
    
    model.compile()
    return model

In [ ]:
def createDiscriminator(z_dims, n_dims):
    z_inputs = Input(shape=(z_dims))
    n_inputs = Input(shape=(n_dims))
    inputs = concatenate([z_inputs, n_inputs])
    
    # 150, 100 is arbitrary rn...
    dense = Dense(150)(inputs)
    dense = Dense(100)(dense)
    output = Dense(1)
    
    model = Model(inputs=[z_inputs, n_inputs], outputs=output)
    
    model.compile()
    return model

In [ ]:
enc_optimizer = tf.keras.optimizers.Adam(5e-4)
dec_optimizer = tf.keras.optimizers.Adam(5e-4)
disc_optimizer = tf.keras.optimizers.Adam(5e-4)

## When $P_Z$ is known... (Writing this out for clarity—will not be used)

In [ ]:

# k is num of domains.
# encoders is a list of encoders.
# decoders is list of decoders.
# samples is a K x N array of samples, where the first index is the domain,
# the second index is the # of the sample in that domain.

# TODO IMPORTANT: Currently assuming P_Z is known, but it is NOT. Must alter algorithm as in (3.2) to support unknown P_Z.
def trainAutoencoders(k, encoders, decoders, samples, discriminator, weight=1.0):
    N = samples.shape[0]
    
    for i in range(k):
        encoder = encoders[i]
        decoder = decoders[i]
        while(not isConverged(encoder, decoder)):
            p_Xi_samples = samples[i,:]
            p_Z_samples = projectZ(encoder(samples[]))
            p_Ni_samples = None # TODO Something!
            
            with tf.GradientTape() as enc_tape, tf.GradientTape() as dec_tape, tf.GradientTape() as disc_tape:
                
                reconstruction_loss = reconstructionLoss(p_Xi_samples, encoder, decoder, f_w, weight)
                
                # negative b/c gradient ascent.
                divergence_loss = -divergenceLoss(f_w, encoder, p_Xi_samples, p_Z_samples, p_Ni_samples)
                
            gradients_of_encoder = enc_tape.gradient(reconstruction_loss, encoder.trainable_variables)
            gradients_of_decoder = dec_tape.gradient(reconstruction_loss, decoder.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(divergence_loss, discriminator.trainable_variables)
            

            enc_optimizer.apply_gradients(zip(gradients_of_encoder, encoder.trainable_variables))
            dec_optimizer.apply_gradients(zip(gradients_of_decoder, decoder.trainable_variables))
            disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
            

## When $P_Z$ is unknown...
"A straight-forward approach for learning the latent distribution PZ is to train a regularized autoencoder on data from a
single representative domain. However, such a representation could potentially capture variability that is specific to
that one domain. To learn a more invariant latent representation, we propose the following extension of our autoencoder
framework. The basic idea is to alternate between training
multiple autoencoders until they agree on a latent representation that is effective for their respective domains. This is
particularly relevant for applications to biology; for example, often one is interested in learning a latent representation
that integrates all of the data modalities."

In [ ]:

# k is num of domains.
# encoders is a list of encoders.
# decoders is list of decoders.
# samples is a K x N array of samples, where the first index is the domain,
# the second index is the # of the sample in that domain.

# TODO IMPORTANT: Currently assuming P_Z is known, but it is NOT. Must alter algorithm as in (3.2) to support unknown P_Z.
def trainAutoencoders(k, p_Z_hats, encoders, decoders, samples, discriminator, weight=1.0):
    N = samples.shape[0]
    
    for i in range(k):
        encoder = encoders[i]
        decoder = decoders[i]
        p_Z_hat = p_Z_hats[i]
        while(not isConverged(encoder, decoder)):
            p_Xi_samples = samples[i,:]
            p_Zi_samples = projectZ(encoder(samples[i,:]))
            p_Ni_samples = None # TODO Something!
            
            with tf.GradientTape() as enc_tape, tf.GradientTape() as dec_tape, tf.GradientTape() as disc_tape:
                
                reconstruction_loss = reconstructionLoss(p_Xi_samples, encoder, decoder, f_w, weight)
                
                # negative b/c gradient ascent.
                divergence_loss = -divergenceLoss(f_w, encoder, p_Xi_samples, p_Z_samples, p_Ni_samples)
                
            gradients_of_encoder = enc_tape.gradient(reconstruction_loss, encoder.trainable_variables)
            gradients_of_decoder = dec_tape.gradient(reconstruction_loss, decoder.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(divergence_loss, discriminator.trainable_variables)
            

            enc_optimizer.apply_gradients(zip(gradients_of_encoder, encoder.trainable_variables))
            dec_optimizer.apply_gradients(zip(gradients_of_decoder, decoder.trainable_variables))
            disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
            

## Evaluation with Rouge

In [ ]:
def evaluateOnArticles(articles, encoder, decoder, evaluator):
    translated = decoder(encoder(articles))
    scores = evaluator.get_scores(translated, original)
    
    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        print('\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * results['p'], 'R', 100.0 * results['r'], 'F1', 100.0 * results['f']))

In [ ]:
def evaluate(articles, encoders, decoders):
    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=4,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        apply_avg=False,
                        apply_best=True,
                        alpha=0.5, # Default F1_score
                        weight_factor=1.2,
                        stemming=True)
    
    #TODO Decide which articles are being tested with which styles